### Проект "Выбираем авто выгодно".
### Автор: Алексей Киселев
### Использованы наработки коллег:
### Датасет, использованный для тренировки модели: <a href='https://www.kaggle.com/macsunmood/autoru-parsed-0603-1304'> датасет от macsunmood (спасибо ему ОГРОМНОЕ!!!)</a>
### За основу решения взят <a href='https://www.kaggle.com/abdualimov/atp-final'>ноутбук от SOVIET (также спасибо ему ОГРОМНОЕ!!!</a>

In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import ExtraTreeRegressor
from catboost import CatBoostRegressor

from tqdm import tqdm

In [2]:
RANDOM_SEED = 13  # фиксируем рандомные параметры и размер валидационной выборки выборки
VAL_SIZE   = 0.33
iterations = 6000
learning_rate = 0.05
VERSION = 7

In [3]:
asd_99 = pd.read_csv('data/auto_ru.csv', index_col = 0) # открываем датасет и сразу удаляем дубликаты 
print(asd_99.shape)
asd_99 = asd_99.drop_duplicates().reset_index(drop = True)
print('Размер трейна: ', asd_99.shape, '\n',
      'Названия столбцов трейна: ', ', '.join(asd_99.columns))
a = pd.read_csv('data/test.csv')
print('Размер теста: ', a.shape, '\n',
      'Названия столбцов теста: ', ', '.join(a.columns))
sample_submission = pd.read_csv('data/sample_submission.csv')

(109080, 21)
Размер трейна:  (109080, 21) 
 Названия столбцов трейна:  bodyType, brand, color, fuelType, modelDate, name, numberOfDoors, productionDate, vehicleConfiguration, engineDisplacement, enginePower, description, mileage, Комплектация, Привод, Руль, Владельцы, ПТС, Таможня, Владение, Price
Размер теста:  (3837, 23) 
 Названия столбцов теста:  bodyType, brand, color, fuelType, modelDate, name, numberOfDoors, productionDate, vehicleConfiguration, vehicleTransmission, engineDisplacement, enginePower, description, mileage, Комплектация, Привод, Руль, Состояние, Владельцы, ПТС, Таможня, Владение, id


In [4]:
asd_99['bodyType'].unique()

array(['Седан', 'Универсал 5 дв.', 'Хэтчбек 5 дв. Sportback',
       'Хэтчбек 3 дв.', 'Хэтчбек 5 дв.', 'Кабриолет', 'Седан Long',
       'Купе', 'Лифтбек Sportback', 'Лифтбек', 'Внедорожник 5 дв.',
       'Родстер', 'Кабриолет Roadster', 'Компактвэн',
       'Компактвэн Gran Tourer', 'Лифтбек Gran Turismo',
       'Хэтчбек 3 дв. Compact', 'Седан 2 дв.', 'Лифтбек Gran Coupe',
       'Седан Gran Coupe', 'Купе-хардтоп', 'Родстер Roadster',
       'Седан Type-S', 'Седан US Market', 'Микровэн',
       'Универсал 5 дв. Shuttle', 'Родстер del Sol', 'Минивэн',
       'Компактвэн Spike', 'Внедорожник 3 дв.', 'Пикап Двойная кабина',
       'Минивэн Spada', 'Внедорожник 5 дв. Arctic Trucks',
       'Внедорожник 5 дв. L', 'Хэтчбек 5 дв. Long',
       'Универсал 5 дв. All-Terrain', 'Внедорожник открытый',
       'Минивэн Long', 'Лимузин Pullman', 'Минивэн L', 'Минивэн XL',
       'Минивэн Marco Polo', 'Минивэн Extra Lang', 'Минивэн Lang',
       'Минивэн Extralang', 'Минивэн L2', 'Минивэн L1', 'Мин

In [5]:
re.findall('\d дв.','Внедорожник 5 дв. Long')

['5 дв.']

In [6]:
asd_99[~asd_99['bodyType'].isna()]['bodyType'].apply(lambda x: str(x).split()[0]).unique()

array(['Седан', 'Универсал', 'Хэтчбек', 'Кабриолет', 'Купе', 'Лифтбек',
       'Внедорожник', 'Родстер', 'Компактвэн', 'Купе-хардтоп', 'Микровэн',
       'Минивэн', 'Пикап', 'Лимузин', 'Тарга', 'Седан-хардтоп', 'Фургон'],
      dtype=object)

In [7]:
def param_data(data): # посмотрим на данные
  param = pd.DataFrame({
              'dtypes': data.dtypes.values,
              'nunique': data.nunique().values,
              'isna': data.isna().sum().values,
              'loc[0]': data.loc[0].values,
              }, 
             index = data.loc[0].index)
  return param

In [8]:
pd.concat([param_data(asd_99), param_data(a)], axis=1, sort=False)

,dtypes,nunique,isna,loc[0],dtypes,nunique,isna,loc[0]
bodyType,object,112.0,1.0,Седан,object,11.0,0.0,седан
brand,object,13.0,0.0,AUDI,object,1.0,0.0,BMW
color,object,16.0,0.0,040001,object,15.0,0.0,чёрный
fuelType,object,6.0,0.0,бензин,object,4.0,0.0,дизель
modelDate,float64,70.0,1.0,1990,float64,31.0,0.0,2016
name,object,3490.0,1.0,2.3 MT (133 л.с.),object,384.0,0.0,520d 2.0d AT (190 л.с.)
numberOfDoors,float64,5.0,1.0,4,float64,4.0,0.0,4
productionDate,int64,73.0,0.0,1991,float64,37.0,0.0,2017
vehicleConfiguration,object,4.0,1.0,MECHANICAL,object,99.0,0.0,SEDAN AUTOMATIC 2.0
engineDisplacement,object,431.0,1.0,2.3,object,22.0,0.0,2.0 LTR


In [9]:
def train_(train):
    train = train.drop(['Таможня'], axis = 1)
  # bodyType, оставим только названия кузова как в тесте
    train['bodyType'] = train['bodyType'].apply(lambda x: str(x))
    def bodyType(row):
        for body_type in ['внедорожник 5 дв.', 'седан-хардтоп', 'хэтчбек 5 дв.','внедорожник 3 дв.', 
                      'купе-хардтоп', 'внедорожник открытый','хэтчбек 3 дв.', 'пикап двойная кабина',
                      'пикап полуторная кабина', 'пикап одинарная кабина', 'седан 2 дв.','универсал 5 дв.', 
                      'родстер', 'кабриолет','фургон', 'микровэн','минивэн', 'компактвэн','лифтбек',
                      'купе','тарга', 'седан','лимузин']:
            if row.lower().startswith(body_type):
                return body_type
#     train['bodyType'] = train['bodyType'].apply(lambda x: str(x).split()[0])
  # color, сделаем значения как в тесте
    train['color'] = train['color'].map({'CACECB': 'серебристый', 'FAFBFB':'белый', 'EE1D19':'красный', '97948F':'серый', 
                                     '660099':'пурпурный', '040001':'чёрный','4A2197':'фиолетовый', 
                                     '200204':'коричневый','0000CC':'синий', '007F00':'зелёный', 'C49648':'бежевый',
                                     '22A0F8':'голубой','DEA522':'золотистый','FFD600': 'жёлтый', 'FF8649':'оранжевый',
                                     'FFC0CB':'розовый'})
  # Руль
    train['Руль'] = train['Руль'].map({'LEFT': 'Левый', 'RIGHT': 'Правый'})
  # ПТС
    train['ПТС'] = train['ПТС'].map({'ORIGINAL': 'Оригинал', 'DUPLICATE': 'Дубликат'}).fillna('Оригинал')
  # Владение
    train['Владение'] = train['Владение'].fillna('nodata')
    def months_to_sent(months):
        if months == 1:
            return f'{months} месяц'
        elif 2 <= months <= 4:
            return f'{months} месяца'
        return f'{months} месяцев'
    def years_to_sent(years):
        if 11 <= years <= 14 or 5 <= years%10 <= 9 or years%10 == 0:
            return f'{years} лет'
        elif years%10 == 1:
            return f'{years} год'
        elif 2 <= years%10 <= 4:
            return f'{years} годa'
    def tenure(row):
        row = re.findall('\d+',row)
        if row != []:
            years = 2020 - (int(row[0])+1)
            months = 2 +(12 - int(row[1]))
            if years < 0:
                return months_to_sent(int(row[1]))
            elif years == 0 and months < 12:
                return months_to_sent(months)
            elif years >= 0 and months == 12:
                return years_to_sent(years + 1)
            elif years >= 0 and months > 12:
                return years_to_sent(years + 1)+' и '+months_to_sent(months - 12)
            elif years > 0 and months < 12:
                return years_to_sent(years)+' и '+months_to_sent(months)
            return None
    train['Владение'] = train['Владение'].apply(tenure)
    return train

In [10]:
def test_(test):
    
  # удалим ненужные столбцы
    test = test.drop(['id','Состояние', 'Таможня', 'vehicleTransmission'], axis = 1)
#     test['bodyType'] = test['bodyType'].apply(lambda x: str(x).split()[0])
  # приравняем к единому числовому варианту
    for feature in ['modelDate', 'numberOfDoors', 'productionDate', 'mileage']:
        test[feature] = test[feature].astype('int64')
  # vehicleConfiguration, выведем только трансмиссию
    test['vehicleConfiguration'] = test['vehicleConfiguration'].apply(lambda x: x.split()[1])
    test['engineDisplacement'] = test['engineDisplacement'].apply(lambda x: x.split()[0])
  # enginePower, оставим только цифру в enginePower
    test['enginePower'] = test['enginePower'].apply(lambda x: int(x.split()[0]))
  # Владельцы, оставим только цифру
    test['Владельцы'] = test['Владельцы'].apply(lambda x: int(x.split()[0]))
    return test


In [11]:
train = train_(asd_99)
train.dropna(axis = 0, thresh=18,inplace=True)#строки , где слишком много пропущенных значений.
test = test_(a)
pd.concat([param_data(train), param_data(test)], axis=1, sort=False)

,dtypes,nunique,isna,loc[0],dtypes,nunique,isna,loc[0]
bodyType,object,112,0,Седан,object,11.0,0.0,седан
brand,object,13,0,AUDI,object,1.0,0.0,BMW
color,object,16,0,чёрный,object,15.0,0.0,чёрный
fuelType,object,5,0,бензин,object,4.0,0.0,дизель
modelDate,float64,70,0,1990,int64,31.0,0.0,2016
name,object,3490,0,2.3 MT (133 л.с.),object,384.0,0.0,520d 2.0d AT (190 л.с.)
numberOfDoors,float64,5,0,4,int64,4.0,0.0,4
productionDate,int64,73,0,1991,int64,37.0,0.0,2017
vehicleConfiguration,object,4,0,MECHANICAL,object,3.0,0.0,AUTOMATIC
engineDisplacement,object,431,0,2.3,object,22.0,0.0,2.0


In [12]:
train.Price=train.Price.apply(lambda x: np.log(x))

In [13]:
train['sample'] = 1
test['sample'] = 0
test['Price'] = 0
data = test.append(train, sort=False).reset_index(drop=True)
param_data(data)


,dtypes,nunique,isna,loc[0]
bodyType,object,123,0,седан
brand,object,13,0,BMW
color,object,16,0,чёрный
fuelType,object,5,0,дизель
modelDate,float64,70,0,2016
name,object,3506,0,520d 2.0d AT (190 л.с.)
numberOfDoors,float64,5,0,4
productionDate,int64,73,0,2017
vehicleConfiguration,object,4,0,AUTOMATIC
engineDisplacement,object,433,0,2.0


In [14]:
data['description'] = data['description'].fillna('[]')
data['description_len'] = data['description'].apply(lambda x: len(x.split()))
data['description_word'] = data['description'].apply(lambda x: [str(i).lower() for i in x.split()])

In [15]:
data['description_word'][300]

['продаю',
 'свой',
 'автомобиль.срок',
 'владения',
 'с',
 '2007',
 'года,',
 'птс',
 '-оригинал,',
 'пробег',
 'реальный.',
 'была',
 'смена',
 'владельца',
 'в',
 'семье.',
 'на',
 'все',
 'вопросы',
 'отвечу',
 'по',
 'телефону.',
 'салоны',
 'и',
 'перекупщиков',
 'просьба',
 'не',
 'беспокоить.торг.']

In [16]:
data['leather']= data['description_word'].apply(lambda x: 
                                                1 if ('темный' and 'салон') in x else 0)
data['carter']= data['description_word'].apply(lambda x: 
                                                1 if ('защита' and 'картера') in x else 0)
data['ABS']= data['description_word'].apply(lambda x: 
                                                1 if ('антиблокировочная' and 'система') in x else 0)
data['airbags']= data['description_word'].apply(lambda x: 
                                                1 if ('подушки' and 'безопасности') in x else 0)
data['immob']= data['description_word'].apply(lambda x: 
                                                1 if ('иммобилайзер') in x else 0)
data['central_locking']= data['description_word'].apply(lambda x: 
                                                1 if ('центральный' and 'замок') in x else 0)
data['on_board_computer']= data['description_word'].apply(lambda x: 
                                                1 if ('бортовой' and 'компьютер') in x else 0)
data['cruise_control']= data['description_word'].apply(lambda x: 
                                                1 if ('круиз-контроль') in x else 0)
data['climat_control']= data['description_word'].apply(lambda x: 
                                                1 if ('климат-контроль') in x else 0)
data['multi_rudder']= data['description_word'].apply(lambda x: 
                                                1 if ('мультифункциональный' and 'руль') in x else 0)
data['power_steering']= data['description_word'].apply(lambda x: 
                                                1 if ('гидроусилитель' or 'гидро' or 'усилитель' and 'руля') in x else 0)
data['light_and_rain_sensors']= data['description_word'].apply(lambda x: 
                                                1 if ('датчики' and 'света' and 'дождя') in x else 0)
data['сarbon_body_kits']= data['description_word'].apply(lambda x: 
                                                1 if ('карбоновые' and 'обвесы') in x else 0)
data['rear_diffuser_rkp']= data['description_word'].apply(lambda x: 
                                                1 if ('задний' and 'диффузор') in x else 0)
data['door_closers']= data['description_word'].apply(lambda x: 
                                                1 if ('доводчики' and 'дверей') in x else 0)
data['rear_view_camera']= data['description_word'].apply(lambda x: 
                                                1 if ('камера' or 'видеокамера' and 'заднего' and 'вида') in x else 0)
data['amg']= data['description_word'].apply(lambda x: 
                                                1 if ('amg') in x else 0)
data['bi_xenon_headlights']= data['description_word'].apply(lambda x: 
                                                1 if ('биксеноновые' and 'фары') in x else 0)
data['from_salon']= data['description_word'].apply(lambda x: 
                                                1 if ('рольф' or 'панавто' or 'дилер' or 'кредит' or 'ликвидация') in x else 0)
data['alloy_wheels']= data['description_word'].apply(lambda x: 
                                                1 if ('легкосплавные' or 'колесные' or 'диски') in x else 0)
data['parking_sensors']= data['description_word'].apply(lambda x: 
                                                1 if ('парктроник' or 'парктронник') in x else 0)
data['dents']= data['description_word'].apply(lambda x: 
                                                1 if ('вмятины' or 'вмятина' or 'царапина' or 'царапины' or 'трещина') in x else 0)
data['roof_with_panoramic_view']= data['description_word'].apply(lambda x: 
                                                1 if ('панорамная' and 'крыша') in x else 0)

In [17]:
vectorizer = CountVectorizer()
text_feat = vectorizer.fit_transform(data['description'])
data['mean'] = text_feat.mean(axis=1)
data['sum'] = text_feat.sum(axis=1)

In [18]:
data['name_1'] = data['name'].apply(lambda x: str(x).split()[0])

In [19]:
spis_eng = [str(i) for i in data['engineDisplacement'].unique()]


In [20]:
def in_eng_duspl(x):
    if x in spis_eng:
        return None
    else:
        return x

In [21]:
def engineDisplacement(row):
    row = str(row)
    engine = re.findall('\d\.\d', row)
    if engine == []:
        return None
    return float(engine[0])

In [22]:
data['engineDisplacement'] = data['name'].apply(engineDisplacement)

In [23]:
data['Владение'].value_counts()

2 месяца               747
3 месяца               714
1 год и 2 месяца       677
1 год и 4 месяца       660
1 год и 3 месяца       648
                      ... 
18 лет и 8 месяцев       1
24 годa и 9 месяцев      1
24 годa и 7 месяцев      1
21 год и 1 месяц         1
21 год и 5 месяцев       1
Name: Владение, Length: 322, dtype: int64

In [24]:
data['Владение_is_none'] = pd.isna(data['Владение']).astype('uint8')
data['Владельцы'] = data['Владельцы'].fillna(3.0)
data['engineDisplacement'] = data['engineDisplacement'].fillna(2.0) 

In [25]:
def num_of_months(row):
    if pd.notnull(row):
        list_ownership = row.split()
        if len(list_ownership) == 2:
            if list_ownership[1] in ['год', 'года', 'лет']:
                return int(list_ownership[0])*12
            return int(list_ownership[0])
        return int(list_ownership[0])*12 + int(list_ownership[3])

In [26]:
data['month_ownership'] = data['Владение'].apply(num_of_months)


In [27]:
data['month_ownership'].fillna(data['month_ownership'].median(),inplace=True)

In [28]:
data.drop(['description', 'Комплектация', 'name', 'description_word', 'Владение'], axis = 1, inplace = True)
print('Количество пропусков: ', data.isna().sum().sum())

Количество пропусков:  0


In [29]:
data = pd.get_dummies(data, columns = ['bodyType', 'brand', 'color', 'fuelType', 'vehicleConfiguration',
                                           'Привод', 'Руль', 'Владельцы', 'ПТС', 'engineDisplacement', 'name_1'], dummy_na=True)

In [30]:
data['modelDate'] = 2020 - data['modelDate']
data['productionDate'] = 2020 - data['productionDate']
data['prod-model'] = data['productionDate'] - data['modelDate']

In [31]:
train_data = data.query('sample==1').drop('sample', axis = 1)
test_data = data.query('sample==0').drop(['sample', 'Price'], axis = 1)

train_data = data.query('sample==1').drop('sample', axis = 1) # отделяем трейновые и тестовые данные
test_data = data.query('sample==0').drop(['sample', 'Price'], axis = 1).reset_index(drop=True)

In [32]:
X = train_data.drop(['Price'], axis=1).reset_index(drop=True)
y = train_data['Price'].values # target

In [33]:
X.head()

,modelDate,numberOfDoors,productionDate,enginePower,mileage,description_len,leather,carter,ABS,airbags,...,name_1_part-time,name_1_sDrive18i,name_1_sDrive20i,name_1_tiptronic,name_1_ultra,name_1_xDrive18d,name_1_xDrive20d,name_1_xDrive20i,name_1_nan,prod-model
0,30.0,4.0,29,133.0,10000,99,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0
1,32.0,4.0,31,115.0,300000,12,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0
2,30.0,5.0,29,133.0,205636,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0
3,30.0,4.0,29,133.0,450000,45,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0
4,30.0,4.0,29,133.0,275250,19,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0


In [ ]:
models = [RandomForestRegressor(n_estimators = 250,random_state = RANDOM_SEED, n_jobs = -1, verbose = 1),
          BaggingRegressor(ExtraTreeRegressor(random_state=RANDOM_SEED), random_state=RANDOM_SEED, n_jobs=-1),
          CatBoostRegressor(iterations=iterations, learning_rate=learning_rate, eval_metric='MAPE', random_state=RANDOM_SEED, verbose=1)]

def stacking_model_predict(models, X, y, test_data, sample_submission):
    for model_ in tqdm(models):
        model_.fit(X, y)
        pred_subm = model_.predict(test_data)
        sample_submission[str(model_)[:6]] = pred_subm
        sample_submission[str(model_)[:6]] = sample_submission[str(model_)[:6]].apply(lambda x: np.exp(x))
        sample_submission[str(model_)[:6]] = sample_submission[str(model_)[:6]].apply(lambda x: round(x/1000)*1000)
    sample_submission['price'] = sample_submission.iloc[:,2:].mean(axis=1)
    sample_submission[['id', 'price']].to_csv(f'submission_v{VERSION}.csv', index=False)
    sample_submission.head(10)

stacking_model_predict(models, X, y, test_data, sample_submission)

  0%|          | 0/3 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   28.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  3.5min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 176 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Done 250 out of 250 | elapsed:    0.1s finished
 33%|███▎      | 1/3 [03:29<06:58, 209.39s/it]

In [ ]:
sample_submission